In [22]:
# abstract type State end
abstract type Gate end

In [23]:
mutable struct OneQubitGate <: Gate
    item::Array{Complex, 2}
    target::Int
    worldline::Int
end

mutable struct TwoQubitGate <: Gate
    item::Array{Complex, 2}
    control::Int
    target::Int
    worldline_c::Int
    worldline_t::Int
end

mutable struct State
    item::Array{Complex, 1}
    target::Int
end

In [68]:
Hmat = 1 / sqrt(2) * [1 1; 1 -1]
CZmat = [1 1; 1 -1]

2×2 Array{Int64,2}:
 1   1
 1  -1

In [262]:
function merge(A::OneQubitGate, B::OneQubitGate, merge_to::Int)
    if merge_to == 1
        item_00 = A.item[1, 1] * B.item[1, 1] + A.item[1, 1] * B.item[1, 2]
        item_01 = A.item[1, 2] * B.item[2, 1] + A.item[1, 2] * B.item[2, 2]
        item_10 = A.item[2, 1] * B.item[1, 1] + A.item[2, 1] * B.item[1, 2]
        item_11 = A.item[2, 2] * B.item[2, 1] + A.item[2, 2] * B.item[2, 2]
        return [item_00 item_01; item_10 item_11], B.item
    elseif merge_to == 2
        item_00 = A.item[1, 1] * B.item[1, 1] + A.item[2, 1] * B.item[1, 1]
        item_01 = A.item[1, 1] * B.item[1, 2] + A.item[2, 1] * B.item[1, 2]
        item_10 = A.item[1, 2] * B.item[2, 1] + A.item[2, 2] * B.item[2, 1]
        item_11 = A.item[1, 2] * B.item[2, 2] + A.item[2, 2] * B.item[2, 2]
        return A.item, [item_00 item_01; item_10 item_11]
    end
end

function merge(A::State, B::OneQubitGate)
    item_00 = A.item[1] * B.item[1, 1]
    item_01 = A.item[1] * B.item[1, 2]
    item_10 = A.item[2] * B.item[2, 1]
    item_11 = A.item[2] * B.item[2, 2]
    return A.item, [item_00 item_01; item_10 item_11]
end

function merge(A::OneQubitGate, B::State)
    item_00 = A.item[1, 1] * B.item[1]
    item_01 = A.item[1, 2] * B.item[2]
    item_10 = A.item[2, 1] * B.item[1]
    item_11 = A.item[2, 2] * B.item[2]
    return [item_00 item_01; item_10 item_11], B.item
end

function merge(A::OneQubitGate, B::TwoQubitGate)
    if A.target == B.target
        item_00 = (A.item[1, 1] + A.item[2, 1]) * B.item[1, 1]
        item_01 = (A.item[1, 2] + A.item[2, 2]) * B.item[1, 2]
        item_10 = (A.item[1, 1] + A.item[2, 1]) * B.item[2, 1]
        item_11 = (A.item[1, 2] + A.item[2, 2]) * B.item[2, 2]
    elseif A.target == B.control
        item_00 = (A.item[1, 1] + A.item[2, 1]) * B.item[1, 1]
        item_01 = (A.item[1, 1] + A.item[2, 1]) * B.item[1, 2]
        item_10 = (A.item[1, 2] + A.item[2, 2]) * B.item[2, 1]
        item_11 = (A.item[1, 2] + A.item[2, 2]) * B.item[2, 2]
    end
    return A.item, [item_00 item_01; item_10 item_11]
end

function merge(A::TwoQubitGate, B::OneQubitGate)
    if A.target == B.target
        item_00 = A.item[1, 1] * B.item[1, 1] + A.item[1, 2] * B.item[2, 1]
        item_01 = A.item[1, 2] * B.item[2, 2] + A.item[1, 1] * B.item[1, 2]
        item_10 = A.item[2, 1] * B.item[1, 1] + A.item[2, 2] * B.item[2, 1]
        item_11 = A.item[2, 2] * B.item[2, 2] + A.item[2, 1] * B.item[1, 2]
    elseif A.control == B.target
        item_00 = A.item[1, 1] * B.item[1, 1] + A.item[2, 1] * B.item[2, 1]
        item_01 = A.item[1, 2] * B.item[1, 1] + A.item[2, 2] * B.item[2, 1]
        item_10 = A.item[2, 1] * B.item[2, 2] + A.item[1, 1] * B.item[1, 2]
        item_11 = A.item[2, 2] * B.item[2, 2] + A.item[1, 2] * B.item[1, 2]
    end
    return [item_00 item_01; item_10 item_11], B.item
end

# function merge(A::TwoQubitGate, B::OneQubitGate)
#     if A.target == B.target
#         item_00 = A.item[1, 1] * (B.item[1, 1] + B.item[1, 2])
#         item_01 = A.item[1, 2] * (B.item[2, 1] + B.item[2, 2])
#         item_10 = A.item[2, 1] * (B.item[1, 1] + B.item[1, 2])
#         item_11 = A.item[2, 2] * (B.item[2, 1] + B.item[2, 2])
#     elseif A.control == B.target
#         item_00 = A.item[1, 1] * (B.item[1, 1] + B.item[1, 2])
#         item_01 = A.item[1, 2] * (B.item[1, 1] + B.item[1, 2])
#         item_10 = A.item[2, 1] * (B.item[2, 1] + B.item[2, 2])
#         item_11 = A.item[2, 2] * (B.item[2, 1] + B.item[2, 2])
#     end
#     return [item_00 item_01; item_10 item_11], B.item
# end

function merge(A::State, B::TwoQubitGate)
    if A.target == B.target
        item_00 = B.item[1, 1] * A.item[1]
        item_01 = B.item[1, 2] * A.item[2]
        item_10 = B.item[2, 1] * A.item[1]
        item_11 = B.item[2, 2] * A.item[2]
    elseif A.target == B.control
        item_00 = A.item[1] * B.item[1, 1]
        item_01 = A.item[1] * B.item[1, 2]
        item_10 = A.item[2] * B.item[2, 1]
        item_11 = A.item[2] * B.item[2, 2]
    end
    return A.item, [item_00 item_01; item_10 item_11]
end

function merge(A::TwoQubitGate, B::State)
    if A.target == B.target
        item_00 = A.item[1, 1] * B.item[1]
        item_01 = A.item[1, 2] * B.item[2]
        item_10 = A.item[2, 1] * B.item[1]
        item_11 = A.item[2, 2] * B.item[2]
    elseif A.control == B.target
        item_00 = B.item[1] * A.item[1, 1]
        item_01 = B.item[1] * A.item[1, 2]
        item_10 = B.item[2] * A.item[2, 1]
        item_11 = B.item[2] * A.item[2, 2]
    end
    return [item_00 item_01; item_10 item_11], B.item
end

function merge(A::TwoQubitGate, B::TwoQubitGate, role::String)
    if role == "tt"
        item_00 = (A.item[1, 1] + A.item[2, 1]) * B.item[1, 1]
        item_01 = (A.item[1, 2] + A.item[2, 2]) * B.item[1, 2]
        item_10 = (A.item[1, 1] + A.item[2, 1]) * B.item[2, 1]
        item_11 = (A.item[1, 2] + A.item[2, 2]) * B.item[2, 2]
    elseif role == "tc"
        item_00 = (A.item[1, 1] + A.item[2, 1]) * B.item[1, 1]
        item_01 = (A.item[1, 1] + A.item[2, 1]) * B.item[1, 2]
        item_10 = (A.item[1, 2] + A.item[2, 2]) * B.item[2, 1]
        item_11 = (A.item[1, 2] + A.item[2, 2]) * B.item[2, 2]
    end
    return A.item, [item_00 item_01; item_10 item_11]
end

function sum(A::Gate)
    return A.item[1, 1] + A.item[1, 2] + A.item[2, 1] + A.item[2, 2]
end

sum (generic function with 1 method)

In [246]:
Input1 = State([1, 0], 1)
Input2 = State([1, 0], 2)
Output1 = State([1, 0], 1)
Output2 = State([1, 0], 2)
H1 = OneQubitGate(Hmat, 1, 1)
H2= OneQubitGate(Hmat, 2, 1)
CZ = TwoQubitGate(CZmat, 1, 2, 2, 2)
H3 = OneQubitGate(Hmat, 1, 3)
H4 = OneQubitGate(Hmat, 2, 3)

OneQubitGate(Complex[0.7071067811865475 + 0.0im 0.7071067811865475 + 0.0im; 0.7071067811865475 + 0.0im -0.7071067811865475 + 0.0im], 2, 3)

In [247]:
function create_graph(_n_qubit, Inputs, Outputs, gates::Array{Gate, 1})
    latest = zeros(_n_qubit)
    edges = []
    for i in 1:len(gates)
        if gates[i] <: OneQubitGate
            idx = gates[i]._target
            if latest[idx] == 0
                push!(edges, ())
            else
                
            end
        end
    end
end

create_graph (generic function with 1 method)

In [248]:
n_qregs = 2
Inputs = [Input1, Input2]
Outputs = [Output1, Output2]
gates = [H1, H2, CZ, H3, H4]

egdes = [
    (Inputs[1], gates[1]),
    (Inputs[2], gates[2]),
    (gates[4], Outputs[1]),
    (gates[5], Outputs[2]),
    (gates[1], gates[3]),
    (gates[2], gates[3]),
    (gates[3], gates[4]),
    (gates[3], gates[5]),
]

egdes[1][1].item, egdes[1][2].item = merge(egdes[1][1], egdes[1][2])
egdes[2][1].item, egdes[2][2].item = merge(egdes[2][1], egdes[2][2])
egdes[3][1].item, egdes[3][2].item = merge(egdes[3][1], egdes[3][2])
egdes[4][1].item, egdes[4][2].item = merge(egdes[4][1], egdes[4][2])
egdes[5][1].item, egdes[5][2].item = merge(egdes[5][1], egdes[5][2])
egdes[6][1].item, egdes[6][2].item = merge(egdes[6][1], egdes[6][2])
egdes[7][1].item, egdes[7][2].item = merge(egdes[7][1], egdes[7][2])
egdes[8][1].item, egdes[8][2].item = merge(egdes[8][1], egdes[8][2])
sum(egdes[8][1])

0.4999999999999999 + 0.0im

### 例1 

In [264]:
for i in 1:2^2
    bin_string = bitstring(UInt8(i-1))
    
    in1 = Int(bin_string[7]) - 48
    in2 = Int(bin_string[8]) - 48
    Input1 = State([1, 0], 1)
    Input2 = State([1, 0], 2)
    Output1 = State([1-in1, in1], 1)
    Output2 = State([1-in2, in2], 2)
    H1 = OneQubitGate(Hmat, 1, 1)
    H2= OneQubitGate(Hmat, 2, 1)
    CZ = TwoQubitGate(CZmat, 1, 2, 2, 2)
    H3 = OneQubitGate(Hmat, 1, 3)
    H4 = OneQubitGate(Hmat, 2, 3)

    n_qregs = 2
    Inputs = [Input1, Input2]
    Outputs = [Output1, Output2]
    gates = [H1, H2, CZ, H3, H4]
    line = [
                [gates[1], gates[3], gates[4]],
                [gates[2], gates[3], gates[5]],
                ]

    for idx in 1:length(line)
        Inputs[idx].item, line[idx][1].item = merge(Inputs[idx], line[idx][1])
        line[idx][length(line[idx])].item, Outputs[idx].item = merge(line[idx][length(line[idx])], Outputs[idx])
        curr = line[idx][1]
        for timestep in 2:length(line[idx])
            curr.item, line[idx][timestep].item = merge(curr, line[idx][timestep])
            if typeof(curr) == OneQubitGate || typeof(curr) == State
                curr = line[idx][timestep]
            end
        end
    end
    println(sum(gates[3]))
end

0.49999999999999983 + 0.0im
0.49999999999999983 + 0.0im
0.49999999999999983 + 0.0im
-0.49999999999999983 + 0.0im


In [289]:
for i in 1:2^2
    bin_string = bitstring(UInt8(i-1))
    
    in1 = Int(bin_string[7]) - 48
    in2 = Int(bin_string[8]) - 48
    Input1 = State([1, 0], 1)
    Input2 = State([1, 0], 2)
    Output1 = State([1-in1, in1], 1)
    Output2 = State([1-in2, in2], 2)
    H1 = OneQubitGate(Hmat, 1, 1)
    H2= OneQubitGate(Hmat, 2, 1)
    CZ = TwoQubitGate(CZmat, 1, 2, 2, 2)
    H3 = OneQubitGate(Hmat, 1, 3)
    H4 = OneQubitGate(Hmat, 2, 3)

    n_qregs = 2
    Inputs = [Input1, Input2]
    Outputs = [Output1, Output2]
    gates = [H1, H2, CZ, H3, H4]
    line = [
                [gates[1], gates[3], gates[4]],
                [gates[2], gates[3], gates[5]],
                ]

    for idx in 1:length(line)
        Inputs[idx].item, line[idx][1].item = merge(Inputs[idx], line[idx][1])
        # line[idx][length(line[idx])].item, Outputs[idx].item = merge(line[idx][length(line[idx])], Outputs[idx])
        curr = line[idx][1]
        for timestep in 2:length(line[idx])
            curr.item, line[idx][timestep].item = merge(curr, line[idx][timestep])
            if typeof(curr) == OneQubitGate || typeof(curr) == State
                curr = line[idx][timestep]
            end
        end
        curr.item, Outputs[idx].item = merge(curr, Outputs[idx])
    end
    println(sum(gates[3]))
end

0.49999999999999983 + 0.0im
0.49999999999999983 + 0.0im
0.49999999999999983 + 0.0im
-0.49999999999999983 + 0.0im


### 例2

In [265]:
for i in 1:2^2
    bin_string = bitstring(UInt8(i-1))
    
    in1 = Int(bin_string[7]) - 48
    in2 = Int(bin_string[8]) - 48
    Input1 = State([1, 0], 1)
    Input2 = State([1, 0], 2)
    Output1 = State([1-in1, in1], 1)
    Output2 = State([1-in2, in2], 2)
    CZ = TwoQubitGate(CZmat, 1, 2, 2, 2)
    H1 = OneQubitGate(Hmat, 1, 3)
    H2 = OneQubitGate(Hmat, 2, 3)

    n_qregs = 2
    Inputs = [Input1, Input2]
    Outputs = [Output1, Output2]
    gates = [CZ, H1, H2]
    line = [
                [gates[1], gates[2]],
                [gates[1], gates[3]],
                ]

    for idx in 1:length(line)
        Inputs[idx].item, line[idx][1].item = merge(Inputs[idx], line[idx][1])
        line[idx][length(line[idx])].item, Outputs[idx].item = merge(line[idx][length(line[idx])], Outputs[idx])
        curr = line[idx][1]
        for timestep in 2:length(line[idx])
            curr.item, line[idx][timestep].item = merge(curr, line[idx][timestep])
            if typeof(curr) == OneQubitGate || typeof(curr) == State
                curr = line[idx][timestep]
            end
        end
    end

    println(sum(gates[1]))
end

0.4999999999999999 + 0.0im
0.4999999999999999 + 0.0im
0.4999999999999999 + 0.0im
0.4999999999999999 + 0.0im


### 例3

In [284]:
for i in 1:2^2
    bin_string = bitstring(UInt8(i-1))
    
    in1 = Int(bin_string[7]) - 48
    in2 = Int(bin_string[8]) - 48
    Input1 = State([1, 0], 1)
    Input2 = State([1, 0], 2)
    Output1 = State([1-in1, in1], 1)
    Output2 = State([1-in2, in2], 2)
    CZ = TwoQubitGate(CZmat, 1, 2, 2, 2)
    H1 = OneQubitGate(Hmat, 1, 1)
    H2 = OneQubitGate(Hmat, 2, 1)

    n_qregs = 2
    Inputs = [Input1, Input2]
    Outputs = [Output1, Output2]
    gates = [H1, H2, CZ]
    line = [
                [gates[1], gates[3]],
                [gates[2], gates[3]],
                ]

    for idx in 1:length(line)
        Inputs[idx].item, line[idx][1].item = merge(Inputs[idx], line[idx][1])
        line[idx][length(line[idx])].item, Outputs[idx].item = merge(line[idx][length(line[idx])], Outputs[idx])
        curr = line[idx][1]
        for timestep in 2:length(line[idx])
            curr.item, line[idx][timestep].item = merge(curr, line[idx][timestep])
            if typeof(curr) == OneQubitGate || typeof(curr) == State
                curr = line[idx][timestep]
            end
        end
    end
    println(sum(gates[3]))
end

0.4999999999999999 + 0.0im
0.4999999999999999 + 0.0im
0.4999999999999999 + 0.0im
-0.4999999999999999 + 0.0im


### 例4

In [288]:
for i in 1:2^2
    bin_string = bitstring(UInt8(i-1))
    
    out1 = Int(bin_string[7]) - 48
    out2 = Int(bin_string[8]) - 48
    Input1 = State([1, 0], 1)
    Input2 = State([1, 0], 2)
    Output1 = State([1-out1, out1], 1)
    Output2 = State([1-out2, out2], 2)
    H1 = OneQubitGate(Hmat, 1, 1)
    H2 = OneQubitGate(Hmat, 2, 1)
    CZ1 = TwoQubitGate(CZmat, 1, 2, 2, 2)
    H3 = OneQubitGate(Hmat, 1, 1)
    H4 = OneQubitGate(Hmat, 2, 1)
    CZ2 = TwoQubitGate(CZmat, 1, 2, 2, 2)
    H5 = OneQubitGate(Hmat, 1, 1)
    H6 = OneQubitGate(Hmat, 2, 1)

    n_qregs = 2
    Inputs = [Input1, Input2]
    Outputs = [Output1, Output2]
    gates = [H1, H2, CZ1, H3, H4, CZ2, H5, H6]
    line = [
                [gates[1], gates[3], gates[4], gates[6], gates[7]],
                [gates[2], gates[3], gates[5], gates[6], gates[8]],
                ]

    for idx in 1:length(line)
        Inputs[idx].item, line[idx][1].item = merge(Inputs[idx], line[idx][1])
        #line[idx][length(line[idx])].item, Outputs[idx].item = merge(line[idx][length(line[idx])], Outputs[idx])
        curr = line[idx][1]
        # 1 qubitずつ処理
        for timestep in 2:length(line[idx])
            if typeof(curr) == TwoQubitGate && typeof(line[idx][timestep]) == TwoQubitGate
                curr = line[idx][timestep]
            else
                curr.item, line[idx][timestep].item = merge(curr, line[idx][timestep])
                if typeof(curr) == OneQubitGate || typeof(curr) == State
                    curr = line[idx][timestep]
                end
            end
            #println(curr)
        end
        curr.item, Outputs[idx].item = merge(curr, Outputs[idx])
    end
    println(gates[3], gates[6])
    
    a = Index(2,"a")
    b = Index(2,"b")

    A = ITensor(ComplexF64 , a,b)
    B = ITensor(ComplexF64 , a,b)

    A[a(1), b(1)] = gates[3].item[1, 1]
    A[a(1), b(2)] = gates[3].item[1, 2]
    A[a(2), b(1)] = gates[3].item[2, 1]
    A[a(2), b(2)] = gates[3].item[2, 2]
    B[a(1), b(1)] = gates[6].item[1, 1]
    B[a(1), b(2)] = gates[6].item[1, 2]
    B[a(2), b(1)] = gates[6].item[2, 1]
    B[a(2), b(2)] = gates[6].item[2, 2]
    C = A*B
    println(C)
end

TwoQubitGate(Complex[0.49999999999999983 + 0.0im 0.49999999999999983 + 0.0im; 0.49999999999999983 + 0.0im -0.49999999999999983 + 0.0im], 1, 2, 2, 2)TwoQubitGate(Complex[0.9999999999999998 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], 1, 2, 2, 2)
ITensor ord=0
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}
 0-dimensional
0.4999999999999997 + 0.0im
TwoQubitGate(Complex[0.49999999999999983 + 0.0im 0.49999999999999983 + 0.0im; 0.49999999999999983 + 0.0im -0.49999999999999983 + 0.0im], 1, 2, 2, 2)TwoQubitGate(Complex[0.0 + 0.0im 0.9999999999999998 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], 1, 2, 2, 2)
ITensor ord=0
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}
 0-dimensional
0.4999999999999997 + 0.0im
TwoQubitGate(Complex[0.49999999999999983 + 0.0im 0.49999999999999983 + 0.0im; 0.49999999999999983 + 0.0im -0.49999999999999983 + 0.0im], 1, 2, 2, 2)TwoQubitGate(Complex[0.0 + 0.0im 0.0 + 0.0im; 0.9999999999999998 + 0.0im -0.0 + 0.0im], 1, 2, 2, 2)
ITensor ord=0
NDTensors.De

In [200]:
a = Index(2,"a")
b = Index(2,"b")

A = ITensor(ComplexF64 , a,b)
B = ITensor(ComplexF64 , a,b)

A[a(1), b(1)] = gates[3].item[1, 1]
A[a(1), b(2)] = gates[3].item[1, 2]
A[a(2), b(1)] = gates[3].item[2, 1]
A[a(2), b(2)] = gates[3].item[2, 2]
B[a(1), b(1)] = gates[6].item[1, 1]
B[a(1), b(2)] = gates[6].item[1, 2]
B[a(2), b(1)] = gates[6].item[2, 1]
B[a(2), b(2)] = gates[6].item[2, 2]

C = A*B

ITensor ord=0
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}

In [212]:
println(C)

ITensor ord=0
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}
 0-dimensional
0.4999999999999997 + 0.0im


In [198]:
a = Index(2,"a")
b = Index(2,"b")
c = Index(2,"c")

# Define 3 order 2 tensors (matrices)
Z = ITensor(ComplexF64 , a,b)
X = ITensor(ComplexF64 , b,c)
Y = ITensor(ComplexF64 , b,c)

# Set some elements
Z[a(1), b(1)] = 1.0 + 0im
Z[a(2), b(2)] = -1.0 + 0im

X[b(1), c(2)] = 1.0 + 1im
X[b(2), c(1)] = 1.0

Y[b(1), c(1)] = 1.0
Y[b(2), c(2)] = 1.0

# Operations with tensors
R = Z * X

S = Y + X

T = Y - X

ITensor ord=2 (dim=2|id=67|"b") (dim=2|id=485|"c")
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}

In [199]:
println("Z =\n", Z, "\n")
println("X =\n", X, "\n")
println("Y =\n", Y, "\n")
println("R = Z * X =\n", R, "\n")
println("S = Y + X =\n", S, "\n")
println("S = Y - X =\n", T, "\n")

Z =
ITensor ord=2
Dim 1: (dim=2|id=105|"a")
Dim 2: (dim=2|id=67|"b")
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}
 2×2
 1.0 + 0.0im   0.0 + 0.0im
 0.0 + 0.0im  -1.0 + 0.0im

X =
ITensor ord=2
Dim 1: (dim=2|id=67|"b")
Dim 2: (dim=2|id=485|"c")
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}
 2×2
 0.0 + 0.0im  1.0 + 1.0im
 1.0 + 0.0im  0.0 + 0.0im

Y =
ITensor ord=2
Dim 1: (dim=2|id=67|"b")
Dim 2: (dim=2|id=485|"c")
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}
 2×2
 1.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  1.0 + 0.0im

R = Z * X =
ITensor ord=2
Dim 1: (dim=2|id=105|"a")
Dim 2: (dim=2|id=485|"c")
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}
 2×2
  0.0 + 0.0im  1.0 + 1.0im
 -1.0 + 0.0im  0.0 + 0.0im

S = Y + X =
ITensor ord=2
Dim 1: (dim=2|id=67|"b")
Dim 2: (dim=2|id=485|"c")
NDTensors.Dense{Complex{Float64},Array{Complex{Float64},1}}
 2×2
 1.0 + 0.0im  1.0 + 1.0im
 1.0 + 0.0im  1.0 + 0.0im

S = Y - X =
ITensor ord=2
Dim 1: (dim=2|id=6

In [282]:
Input1 = State([1, 0], 1)
Input2 = State([1, 0], 2)
Output1 = State([1, 0], 1)
Output2 = State([1, 0], 2)
H1 = OneQubitGate(Hmat, 1, 1)
H2= OneQubitGate(Hmat, 2, 1)
CZ = TwoQubitGate(CZmat, 1, 2, 2, 2)
H3 = OneQubitGate(Hmat, 1, 3)
H4 = OneQubitGate(Hmat, 2, 3)

gates = [H1, H2]
lines = [[H1], [H2]]
curr = lines[1][1]
println(gates[1])
curr.item = [0 0; 0 0]
println(gates[1])

OneQubitGate(Complex[0.7071067811865475 + 0.0im 0.7071067811865475 + 0.0im; 0.7071067811865475 + 0.0im -0.7071067811865475 + 0.0im], 1, 1)
OneQubitGate(Complex[0 + 0im 0 + 0im; 0 + 0im 0 + 0im], 1, 1)
